<a href="https://colab.research.google.com/github/sheekhaV/Agentic_AI_experiments/blob/main/tool_calling_demo_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool Calling with LangChain and Groq

This notebook demonstrates how to use **tool calling** (also known as function calling) with LangChain and Groq's LLaMA model.

## What is Tool Calling?

Tool calling allows an LLM to:
1. Recognize when it needs to use an external function/tool
2. Extract the right parameters from the user's query
3. Call the tool and use the result in its response

This is powerful because LLMs can now interact with external systems, APIs, and perform computations they couldn't do on their own.

## Step 1: Install Dependencies

First, let's install the required packages.

In [ ]:
!pip install langchain langchain-groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.1 MB/s eta 0:00:00


## Step 2: Load Environment Variables

We load the API key from the `.env` file. Make sure your `.env` file contains:
```
GROQ_API_KEY=your_api_key_here
```

In [ ]:
import getpass
import os

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass.getpass('Enter your GROQ API key:')

Enter your GROQ API key:··········


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Verify the API key is loaded
if os.getenv("GROQ_API_KEY"):
    print("GROQ_API_KEY loaded successfully!")
else:
    print("Warning: GROQ_API_KEY not found. Please check your .env file.")

GROQ_API_KEY loaded successfully!


## Step 3: Define a Tool

A **tool** is simply a Python function decorated with `@tool`. The docstring becomes the tool's description that the LLM uses to understand when to call it.

In [ ]:
import math
from langchain.tools import tool

@tool
def square_root(number: float) -> float:
    """
    Calculate the square root of a given number.
    Use this tool when the user asks for the square root of any number.

    Args:
        number: The number to calculate the square root of. Must be non-negative.

    Returns:
        The square root of the input number.
    """
    if number < 0:
        return "Error: Cannot calculate square root of a negative number."
    return math.sqrt(number)

# Test the tool directly
print(f"Square root of 16: {square_root.invoke({'number': 16})}")
print(f"Square root of 2: {square_root.invoke({'number': 2})}")

Square root of 16: 4.0
Square root of 2: 1.4142135623730951


## Step 4: Initialize the Groq LLM

We use Groq's `llama-3.3-70b-versatile` model which supports tool calling.

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0  # Use 0 for deterministic responses
)

print("Groq LLM initialized!")

Groq LLM initialized!


## Step 5: Bind Tools to the LLM

We bind our tool(s) to the LLM. This tells the model what tools are available.

In [ ]:
# Create a list of tools
tools = [square_root]

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print("Tools bound to LLM!")
print(f"Available tools: {[t.name for t in tools]}")

Tools bound to LLM!
Available tools: ['square_root']


## Step 6: Test Tool Calling

Now let's see the LLM decide when to call the tool. First, let's see what happens when we ask a question that requires the tool.

In [ ]:
from langchain_core.messages import HumanMessage

# Ask a question that requires the square root tool
response = llm_with_tools.invoke([HumanMessage(content="What is the square root of 144?")])

print("Response type:", type(response))
print("\nContent:", response.content)
print("\nTool calls:", response.tool_calls)

Response type: <class 'langchain_core.messages.ai.AIMessage'>

Content: 

Tool calls: [{'name': 'square_root', 'args': {'number': 144}, 'id': '5tkz73470', 'type': 'tool_call'}]


Notice that the LLM returns `tool_calls` - it identified that it needs to use our `square_root` tool!

## Step 7: Execute the Tool and Get Final Response

Now we need to:
1. Execute the tool with the extracted arguments
2. Send the result back to the LLM
3. Get the final response

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

def chat_with_tools(user_message: str):
    """Complete flow: user question -> tool call -> final answer"""

    messages = [HumanMessage(content=user_message)]

    # Step 1: Get initial response from LLM
    response = llm_with_tools.invoke(messages)
    messages.append(response)

    # Step 2: Check if the LLM wants to call any tools
    if response.tool_calls:
        print(f"LLM decided to call tool(s): {[tc['name'] for tc in response.tool_calls]}")

        # Execute each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']

            print(f"  Calling {tool_name} with args: {tool_args}")

            # Execute the tool
            if tool_name == 'square_root':
                result = square_root.invoke(tool_args)

            print(f"  Tool result: {result}")

            # Add tool result to messages
            messages.append(ToolMessage(
                content=str(result),
                tool_call_id=tool_call['id']
            ))

        # Step 3: Get final response with tool results
        final_response = llm_with_tools.invoke(messages)
        return final_response.content

    # No tool calls needed
    return response.content

# Test it!
answer = chat_with_tools("What is the square root of 144?")
print(f"\nFinal Answer: {answer}")

LLM decided to call tool(s): ['square_root']
  Calling square_root with args: {'number': 144}
  Tool result: 12.0

Final Answer: The square root of 144 is 12.0.


## Step 8: More Examples

Let's try a few more examples to see how the LLM handles different queries.

In [ ]:
# Example 1: A harder number
print("Example 1:")
print(chat_with_tools("Can you tell me the square root of 529?"))
print()

Example 1:
LLM decided to call tool(s): ['square_root']
  Calling square_root with args: {'number': 529}
  Tool result: 23.0
The square root of 529 is 23.0.



In [ ]:
# Example 2: Question that doesn't need the tool
print("Example 2:")
print(chat_with_tools("What is 2 + 2?"))
print()

Example 2:
2 + 2 = 4.



In [ ]:
# Example 3: Natural language query
print("Example 3:")
print(chat_with_tools("I need to find the square root of 256 for my homework."))

Example 3:
LLM decided to call tool(s): ['square_root']
  Calling square_root with args: {'number': 256}
  Tool result: 16.0
The square root of 256 is 16.


## Summary

### Key Concepts:

1. **Tool Definition**: Use `@tool` decorator on a function. The docstring is crucial - it tells the LLM when to use the tool.

2. **Binding Tools**: Use `llm.bind_tools([...])` to make tools available to the LLM.

3. **Tool Calling Flow**:
   - User sends message
   - LLM analyzes and decides if a tool is needed
   - If yes, LLM returns `tool_calls` with function name and arguments
   - We execute the tool and send results back
   - LLM generates final response using tool output

4. **When to Use Tools**: Tools are great for:
   - Mathematical calculations
   - Database queries
   - API calls
   - File operations
   - Any external system interaction

### Exercise for Students:

Try adding more tools! For example:
- A `cube_root` tool
- A `power` tool (x^n)
- A `factorial` tool

# Your turn! Add more tools here and update the chat_with_tools function

# @tool
# def cube_root(number: float) -> float:
#     """Calculate the cube root of a number."""
#     pass